# 🌐 Tool Calling con API reali in LangChain

## 🎯 Obiettivo

Permettere a un **LLM** di:

* Interagire con **servizi reali via HTTP**
* Recuperare **informazioni dinamiche e contestuali**
* Utilizzare **API esterne** come strumenti

---

## 🏗️ API fittizia con FastAPI

### 📌 API esposta su `http://localhost:5566`

Due endpoint:

* `/weather/{city}` → dati meteo reali per 3 città
* `/outdoor/{city}` → disponibilità posti a sedere

### 🌤️ Esempi:

```json
# GET /weather/monaco
{ "weather": "Soleggiato, 22°C" }

# GET /outdoor/volonta-del-sole
{ "availability": "Posti a sedere all'aperto disponibili" }
```

---

## 🛠️ Step 1 – Definizione degli strumenti

```python
from langchain.tools import tool
import httpx

@tool
def get_weather(city: str) -> str:
    """Controlla il meteo per una città via API"""
    r = httpx.get(f"http://localhost:5566/weather/{city.lower()}")
    if r.status_code == 200:
        return r.json().get("weather", "Informazioni non disponibili")
    return "Impossibile ottenere il meteo"

@tool
def get_outdoor_seating(city: str) -> str:
    """Verifica la disponibilità dei posti all'aperto via API"""
    r = httpx.get(f"http://localhost:5566/outdoor/{city.lower()}")
    if r.status_code == 200:
        return r.json().get("availability", "Informazioni non disponibili")
    return "Impossibile ottenere disponibilità"
```

---

## 🧠 Step 2 – Creazione della catena con strumenti reali

```python
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
llm_with_tools = llm.bind_tools([get_weather, get_outdoor_seating])
```

---

## 🔁 Step 3 – Esecuzione con gestione manuale della chiamata

### 🧾 Funzione wrapper riutilizzabile

```python
def tool_calling_with_api(prompt: str):
    messages = [{"role": "user", "content": prompt}]
    response = llm_with_tools.invoke(messages)
    messages.append(response)

    tool_calls = response.tool_calls or []

    tool_map = {
        "get_weather": get_weather,
        "get_outdoor_seating": get_outdoor_seating
    }

    for call in tool_calls:
        fn = tool_map.get(call["name"])
        if fn:
            output = fn.invoke(call["args"])
            messages.append({
                "tool_call_id": call["id"],
                "role": "tool",
                "content": output
            })

    return llm_with_tools.invoke(messages)
```

---

## 🧪 Esempi di prompt

### ☀️ Caso 1 – "Volontà del sole"

```python
tool_calling_with_api("Che tempo fa a Volontà del sole? Posso sedermi fuori?")
```

✅ Output:

> “Oggi a Volontà del sole il tempo è soleggiato con 25°C. I posti a sedere all’aperto sono disponibili.”

---

### 🌧️ Caso 2 – "Città della pioggia"

```python
tool_calling_with_api("Che tempo fa a Città della pioggia? Posso sedermi fuori?")
```

✅ Output:

> “Oggi a Città della pioggia piove e ci sono 16°C. I posti a sedere all’aperto non sono disponibili.”

---

### 🧠 Caso 3 – Domanda non legata agli strumenti

```python
tool_calling_with_api("Come si chiama il padre della famiglia Simpson?")
```

✅ Output:

> “Homer Simpson.”

📌 Nessuna chiamata API effettuata (verificabile via log FastAPI).

---

## 📈 Vantaggi del Tool Calling via API

| Vantaggio                     | Descrizione                                            |
| ----------------------------- | ------------------------------------------------------ |
| 🌍 Accesso a dati reali       | Meteo, disponibilità, stock, prezzi, ecc.              |
| 🔌 Integrazione esterna       | Qualsiasi servizio REST può essere connesso facilmente |
| 🧩 Flessibilità               | Strumenti modulari e indipendenti                      |
| 🛡️ Nessun tool → no chiamate | Il LLM decide *quando* usare i tools                   |

---

## ⚠️ Best Practices

* 🔢 **Limita il numero di strumenti**: troppi tool simili = alto rischio errore
* 🧪 **Valida l'output**: gli strumenti devono restituire sempre output coerente
* 📑 **Descrizioni chiare**: la `docstring` è fondamentale per l’accuratezza del matching
* 🔍 **Loggatura attiva**: utile per debug e monitoraggio chiamate reali

---

## ✅ Conclusione

Hai imparato a:

* Costruire strumenti collegati ad API reali via FastAPI
* Eseguire `tool_calling` in modalità multi-tool
* Gestire il ciclo completo: LLM → tool call → API → risposta
* Realizzare un sistema **dinamico**, **modulare** e **potente**

> 🎉 Complimenti! Hai completato **l’intero corso** su come portare le tue applicazioni RAG e LLM **a un livello avanzato** con LangChain.
